In [1]:
import pandas as pd
from pandas import DataFrame,Series
df0=pd.read_csv('d:/lab1data0.csv')
df1=pd.read_csv('d:/lab1data1.csv')
df2=pd.read_csv('d:/lab1data2.csv')
df3=pd.read_csv('d:/lab1data3.csv')
df4=pd.read_csv('d:/lab1data4.csv')
df5=pd.read_csv('d:/lab1data5.csv')
df=[df0,df1,df2,df3,df4,df5]
channel=["全","言情","耽美","百合","女尊","无CP"]
level=[30,50,100,300,500]

In [2]:
#各频道TOP30,TOP100,2019进榜小说平均收藏数
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
avg30=[]
avg100=[]
avg2019=[]
for i in range(6): #channel
    temp=df[i].collected.tolist()
    avg30.append(round(sum(temp[:30])/30))
    avg100.append(round(sum(temp[:100])/100))
    now=df[i]
    temp=now[now['year'].isin([2019])]
    temp=temp.collected.tolist()
    avg2019.append(round(sum(temp)/len(temp)))


def bar_base() -> Bar:
    c = (
        Bar(init_opts=opts.InitOpts(width='900px', height='400px'))
        .add_xaxis(channel)
        .add_yaxis("top30小说平均收藏量",avg30,gap='-100%')
        .add_yaxis("top100小说平均收藏量",avg100)
        .add_yaxis("2019进榜小说平均收藏量",avg2019)
        .set_global_opts(title_opts=opts.TitleOpts(title="各频道平均收藏量"),toolbox_opts=opts.ToolboxOpts())
        
    )
    return c

bar_base().render_notebook()


In [3]:
#各频道视角分布 两组对比 言情vs无cp  耽美vs百合 
from pyecharts.charts import Pie
seq=[1,5,2,3]
index=[]
num=[]
for i in range(4):
    now=df[seq[i]]
    index.append(now.view.value_counts().index)
    num.append(now.view.value_counts())
def pie_multiple_base() -> Pie:
    fn = """
    function(params) {
        if(params.name == '其他')
            return '\\n\\n\\n' + params.name + ' : ' + params.value + '%';
        return params.name + ' : ' + params.value + '%';
    }
    """

    def new_label_opts():
        return opts.LabelOpts(formatter=JsCode(fn), position="center")

    c = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(index[0], num[0])],
            center=["20%", "30%"],
            radius=[60, 80],
        )
        .add(
            "",
            [list(z) for z in zip(index[1], num[1])],
            center=["55%", "30%"],
            radius=[60, 80],
        )
        .add(
            "",
            [list(z) for z in zip(index[2], num[2])],
            center=["20%", "70%"],
            radius=[60, 80],
        )
        .add(
            "",
            [list(z) for z in zip(index[3], num[3])],
            center=["55%", "70%"],
            radius=[60, 80],
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="各频道视角分布",subtitle="第一行 言情vs无cp  第二行 耽美vs百合",pos_left="center"),
            legend_opts=opts.LegendOpts(
                type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"
            ),
            toolbox_opts=opts.ToolboxOpts()
        )
    )
    return c

pie_multiple_base().render_notebook()

In [4]:
#各频道发表时间分布
years=[[],[],[],[],[]]
for i in range(6): #channel
    temp=dict(df[i].year.value_counts())
    count=0
    for y in range(5): #year
        if y==4:
            years[y].append(int(500-count))
        else:
            years[y].append(int(temp[2019-y]))
            count=count+temp[2019-y]
            
#bug1生成条状图时失败,因为'numpy.int64'和int不同，无法用numpy.int64画柱状图
def bar_base1() -> Bar:
    c = (
        Bar(init_opts=opts.InitOpts(width='950px', height='300px'))
        .add_xaxis(channel)
        .add_yaxis("2019年",years[0],stack=True)
        .add_yaxis("2018年",years[1],stack=True)
        .add_yaxis("2017年",years[2],stack=True)
        .add_yaxis("2016年",years[3],stack=True)
        .add_yaxis("2015年及以前",years[4],stack=True)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="各频道top500发表时间分布"))
        
        
    )
    return c

bar_base1().render_notebook()

In [5]:
#小说背景年代分布
temp=df0.booktype.tolist()
for i in range(len(temp)):
    temp[i]=temp[i][6:10]
temp=Series(temp)
temp=dict(temp.value_counts())
era={'古代':0,'近现代':0,'未来':0}
era["古代"]=int(temp["古色古香"]+temp["架空历史"])
era["近现代"]=int(temp["近代现代"])
era["未来"]=int(temp["幻想未来"])

def pie_base() -> Pie:
    c = (
        Pie()
        .add("", [list(z) for z in zip(era.keys(), era.values())])
        .set_global_opts(title_opts=opts.TitleOpts(title="小说背景年代分布"),toolbox_opts=opts.ToolboxOpts())
        .set_series_opts(label_opts=opts.LabelOpts())
    )
    return c

pie_base().render_notebook()

In [6]:
#晋江第一大姓
#本来还有提取前两个字看复姓的，但最后发现现在复姓不吃香了
import re
name1=[]
for i in range(3): #channel
    roles=df[i+1].roles.tolist()
    namelist=[]
    for role in roles:
        names=re.split(r'[，、；/X,]',role)
        namelist.extend(names)
    for name in namelist:
        if name == '' or name==' ':
            namelist.remove(name)
        else:
            name1.append(name[0])  
temp=Series(name1).value_counts()
name=[]
index=[]
for i in range(30):
    index.append(temp.index[i])
    name.append(int(temp[i]))

def bar_toolbox() -> Bar:
    c = (
        Bar()
        .add_xaxis(index)
        .add_yaxis("姓",name)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="晋江人口普查：第一大姓",subtitle="样本为2500个晋江主角名字",pos_left="center"),
            toolbox_opts=opts.ToolboxOpts(),
            legend_opts=opts.LegendOpts(is_show=False)
        )
    )
    return c

bar_toolbox().render_notebook()

In [7]:
#字数分布
llength=["短篇","中篇","长篇","大长篇","超级长篇"]
length=[0,0,0,0,0] #10万 25万 50万 100万
now=df0
for i in range(len(df0)):
    if now.length[i]<=100000:
        length[0]+=1
    elif now.length[i]<=250000:
        length[1]+=1
    elif now.length[i]<=500000:
        length[2]+=1
    elif now.length[i]<=1000000:
        length[3]+=1
    else:
        length[4]+=1

def bar_toolbox() -> Bar:
    c = (
        Bar()
        .add_xaxis(llength)
        .add_yaxis("字数分布",length)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="TOP500小说长度分布",pos_left="center"),
            toolbox_opts=opts.ToolboxOpts(),
            legend_opts=opts.LegendOpts(is_show=False)
        )
    )
    return c

bar_toolbox().render_notebook()


In [8]:
#标签分布
import math
data=[[],[],[]]
for i in range(3):
    now=df[i+1]
    tags=now.tags.tolist()
    taglist=[]
    for j in range(len(tags)):
        if type(tags[j]) is not float :
            taglist.extend(tags[j].split())
    temp=dict(Series(taglist).value_counts()[0:20])
    for name,value in temp.items():
        item={"value":int(value),"name": name}
        data[i].append(item)

In [9]:
#标签分布读者喜好作图
import json
import os

from pyecharts import options as opts
from pyecharts.charts import Page, TreeMap
#可选1，2，3
now_channel=2
def treemap_base() -> TreeMap:
    c = (
        TreeMap(init_opts=opts.InitOpts(bg_color='#FFD322'))
        .add("", data[now_channel-1])
        .set_global_opts(title_opts=opts.TitleOpts(title=channel[now_channel]+"读者喜好",pos_left="center"), toolbox_opts=opts.ToolboxOpts())
    )
    return c
treemap_base().render_notebook()

In [10]:
#全频道top50作者分布
now=df0[0:50]
temp=dict(now.author.value_counts())
data1=[]
for name,value in temp.items():
    item={"value":int(value),"name": name}
    data1.append(item)

def treemap_base() -> TreeMap:
    c = (
        TreeMap(init_opts=opts.InitOpts(bg_color='#FFD322'))
        .add("", data1)
        .set_global_opts(title_opts=opts.TitleOpts(title="全频道top50作者分布",pos_left="center"), toolbox_opts=opts.ToolboxOpts())
    )
    return c
treemap_base().render_notebook()

In [11]:
#文案中常用词分析
import jieba,string
total=[]
for i in range(3):
    total.extend(df[i+1].intro.tolist())
total=str(total)
words = jieba.lcut(total.translate(str.maketrans('', '', string.punctuation))) 
words=Series(words)
#csv输出,人工挑选一波
worddict=dict(words.value_counts()[:500])
df=DataFrame(worddict,index=[0])
df.to_csv("d:/words1.csv",index=False) 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\sanw\AppData\Local\Temp\jieba.cache
Loading model cost 1.119 seconds.
Prefix dict has been built successfully.


In [12]:
#词云
df=pd.read_csv('d:/words.csv')
word=df.word.tolist()
counts=df.counts.tolist()
final=[]
for i in range(len(df.word.tolist())):
    if type(word[i]) is not float:
        final.append((word[i],int(counts[i])))   

from pyecharts.charts import Page, WordCloud
from pyecharts.globals import SymbolType

def wordcloud_base() -> WordCloud:
    c = (
        WordCloud()
        .add("", final, word_size_range=[20, 100])
        .set_global_opts(title_opts=opts.TitleOpts(title="文案词云"),toolbox_opts=opts.ToolboxOpts())
    )
    return c

wordcloud_base().render_notebook()